In [1]:
## Importing helper libraries
import numpy as np
import seaborn as sns
import pandas as pd
import numba as jit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import plotly.express as ply

C:\Users\kkara\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
## Loading dataset
path = 'C:/Users/kkara/Downloads/dataset/'

df_train = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path +'test.csv')

In [3]:
df_train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,376430.000000,166821.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,9.842329,12.668243,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,5.086590,4.125338,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [4]:
df_train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


## Some Data Preprocessing

In [15]:
## Preprocessing train data
df_train['Age']=df_train.Age.map({'0-17':1,'18-25':2,'26-35':3,'36-45':4,'46-50':4,'51-55':5,'55+':6}).astype(np.float)
df_train['Stay_In_Current_City_Years']=df_train['Stay_In_Current_City_Years'].str.extract('(\d+)').astype(np.float)
df_train['Gender']=df_train.Gender.map({'F':0,'M':1}).astype(np.float)
df_train['City_Category']=df_train.City_Category.map({'A':0,'B':1,'C':2}).astype(np.float)
df_train['Product_ID']=df_train['Product_ID'].str.extract('(\d+)').astype(np.float)
df_train['User_ID']=df_train['User_ID']%1000000
#df_train=df_train.fillna(99)

df_train = df_train.fillna(0)


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null float64
Gender                        550068 non-null float64
Age                           550068 non-null float64
Occupation                    550068 non-null int64
City_Category                 550068 non-null float64
Stay_In_Current_City_Years    550068 non-null float64
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null float64
Product_Category_3            550068 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(7), int64(5)
memory usage: 50.4 MB


In [17]:
df_test['Age']=df_test.Age.map({'0-17':1,'18-25':2,'26-35':3,'36-45':4,'46-50':4,'51-55':5,'55+':6}).astype(np.float)
df_test['Stay_In_Current_City_Years']=df_test['Stay_In_Current_City_Years'].str.extract('(\d+)').astype(np.float)
df_test['Gender']=df_test.Gender.map({'F':0,'M':1}).astype(np.float)
df_test['City_Category']=df_test.City_Category.map({'A':0,'B':1,'C':2}).astype(np.float)
df_test['Product_ID']=df_test['Product_ID'].str.extract('(\d+)').astype(np.float)
df_test['User_ID']=df_test['User_ID']%1000000

df_test=df_test.fillna(0)

In [18]:
df_full = pd.concat([df_train,df_test],axis =0)
df_xtrain=df_full.iloc[:550068,:]
df_xtest=df_full.iloc[550068:,:]
df_xtest=df_xtest.drop(['Purchase'],axis=1)

C:\Users\kkara\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




In [19]:
y=df_xtrain.Purchase
x=df_xtrain.drop(['Purchase'],axis=1)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

## Models

In [23]:
# Adaboost Regression

ada_reg = AdaBoostRegressor(random_state=0, n_estimators= 45)
ada_reg.fit(X_train, y_train)
y_pred = ada_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_pred, y_test))
print('Ada rmse: ', rmse)

Ada rmse:  4069.0287751247497


In [24]:
# Decision Tree 

dtree_reg = DecisionTreeRegressor(random_state=0) 
dtree_reg.fit(X_train, y_train)
y_pred = dtree_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_pred, y_test))
print('Decision tree rmse: ', rmse)

Decision tree rmse:  3778.807388988365


In [25]:
# RandomForest 

ran_reg = RandomForestRegressor(max_depth=2, random_state=0)
ran_reg.fit(X_train, y_train)
y_pred = ran_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_pred, y_test))
print('Random tree rmse: ', rmse)

Random tree rmse:  4163.747031944405


In [26]:
# Gradient Boost

grad_reg = GradientBoostingRegressor(n_estimators=40, learning_rate=0.5, random_state=0)
grad_reg.fit(X_train, y_train)
y_pred = grad_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_pred, y_test))
print('Grad rmse: ', rmse)

Grad rmse:  2871.687815358559


In [27]:
# XGB Regressor

xgb_reg = XGBRegressor(learning_rate=0.6, max_depth=6, min_child_weight=40, seed=0)

xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_pred, y_test))

print('XBG rmse: ', rmse)

C:\Users\kkara\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



[12:03:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XBG rmse:  2584.9833404279157
